In [3]:
import pandas as pd
import numpy as np
import io
from sqlalchemy import create_engine
from datetime import datetime
import os
import psycopg2.extras
from psycopg2.pool import ThreadedConnectionPool
import psycopg2 as pg
import psycopg2  
import sys
import warnings 
from io import StringIO
import pyreadstat

import boto3 

#%matplotlib inline  
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.precision',2)

In [5]:
s3_resource = boto3.resource('s3', aws_access_key_id=AK_KC, aws_secret_access_key=SK_KC)
s3_client = boto3.client('s3', aws_access_key_id=AK_KC, aws_secret_access_key=SK_KC)

In [ ]:
for bucket in s3_resource.buckets.all():
    print(bucket.name)
    bucketname=bucket.name

In [ ]:
def get_list_files(Prefix):
    global max_lastdate_file
    try:
        my_bucket = s3_resource.Bucket(bucketname)
        lst_ficheros=[]
        max_lastdate_file=[]
        for my_bucket_object in my_bucket.objects.filter(Prefix= Prefix):
            file_datemodified=my_bucket_object.last_modified.replace(tzinfo = None)
            if file_datemodified>fecha_max:
                if my_bucket_object.key.endswith('gz'):
                    lst_ficheros.append(my_bucket_object.key)
                    file=[my_bucket_object.key, file_datemodified]
                    max_lastdate_file.append(file)
        return max_lastdate_file# lst_ficheros   
    except Exception as e:
        print(f"Error al obtener la lista de archivos: {e}")
        return []    

In [8]:
hora_inicio=(datetime.now())

In [9]:
def get_files_with_extension(folder_path, extension):
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(extension):
                file_list.append(os.path.join(root, file))
    return file_list

In [10]:
folder_path = r'G:\Mi Unidad\Clientes\Kids Corp\2.Proyectos\2.Paneles Encuestas\1.Inputs'
#folder_path = r'G:\My Drive\Clientes\Kids Corp\2.Proyectos\2.Paneles Encuestas\1.Inputs'
#folder_path=os.getcwd()
extension = '.sav'

sav_files = get_files_with_extension(folder_path, extension)

sav_files

['G:\\Mi Unidad\\Clientes\\Kids Corp\\2.Proyectos\\2.Paneles Encuestas\\1.Inputs\\ASKDAR_251323_Ago23_w5_20230914_Latam+Usa_pp3.sav',
 'G:\\Mi Unidad\\Clientes\\Kids Corp\\2.Proyectos\\2.Paneles Encuestas\\1.Inputs\\ASKDAR_251323_Nov23_w7_20231227_Latam+Usa_pp.sav',
 'G:\\Mi Unidad\\Clientes\\Kids Corp\\2.Proyectos\\2.Paneles Encuestas\\1.Inputs\\ASKDAR_251323_Oct23_w6_20231130_Latam+Usa_pp.sav']

In [11]:
# Configuración de la conexión a PostgreSQL
nombre_usuario = 'dbpmconsulting'
contrasena = 'Dbpm2017'
nombre_base_datos = 'postgres'
host = 'scrapping.ciekeymv912n.us-west-2.rds.amazonaws.com'
puerto = '5432'
schema='stg'


In [12]:
def call_cursor():
    global conn, cursor
    conn = psycopg2.connect(host=host, port=puerto, database=nombre_base_datos, user=nombre_usuario, password=contrasena)
    conn.autocommit = True
    cursor = conn.cursor()

In [13]:
def lista_filtrada(df_resultado, lista):
    # Filtrar las columnas del DataFrame
    lista_filtrada = [col for col in df_resultado.columns if col in lista]

    return lista_filtrada

In [14]:
# Define a connect function for PostgreSQL database server
def connect():
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(host=host, port=puerto, database=nombre_base_datos, user=nombre_usuario, password=contrasena)
        conn.autocommit = True
        print("Connection successfully..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)        
        # set the connection to 'None' in case of error
        conn = None
    return conn

In [15]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

In [16]:
# Define function using copy_from() with StringIO to insert the dataframe
def copy_from_dataFile_StringIO(conn, datafrm, table, col_orden):
    call_cursor()
  # save dataframe to an in memory buffer
    buffer = StringIO()
    datafrm[col_orden].to_csv(buffer, header=False, index = False, sep="|", na_rep='NaN')
    buffer.seek(0)
    #conn=connect()
    cursor = conn.cursor()
    cursor.execute("SET search_path TO " + schema)
    try:
        cursor.copy_from(buffer, table, sep="|")
        print("Data inserted using copy_from_datafile_StringIO() successfully....")
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

In [17]:
def call_upsert(fn_upsert):
    try:
        cursor.callproc(fn_upsert)
    except (Exception, psycopg2.DatabaseError) as err:
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()

In [18]:
def completa_cols_resultados(df_resultado, lista):
    pk=lista.copy()
    for columna in pk:
        # Verificar si la columna no existe en el DataFrame
        if columna not in df_resultado.columns:
            # Crear la nueva columna y asignarle un valor predeterminado (por ejemplo, None)
            df_resultado[columna] = None

### Procesamiento de la información

In [19]:
## Lectura del Fichero SPSS 
# Carga en DataFrame y formato columnas
dataframes = []
meta_lst = []
for path in sav_files:
    df_panel, meta = pyreadstat.read_sav(path)
    df_panel.columns=df_panel.columns.str.lower()
    df_panel['respondentid']= df_panel['respondentid'].astype(np.int64)
    df_panel.columns = df_panel.columns.str.replace(' ', '_')
    df_panel.columns = df_panel.columns.str.lower()   
    df_panel['file']=os.path.basename(path)
    dataframes.append(df_panel)
    meta_lst.append(meta)

df_panel_final = pd.concat(dataframes, ignore_index=True)
df_panel_final.shape

(16615, 14612)

In [ ]:
#df_panel_bkp=df_panel_final.copy()

In [ ]:
#df_panel_final=df_panel_bkp.copy()

In [ ]:
## Define las listas de campos por dimension 


lst_pk_respondent=['key','respondentid']
lst_new_vars=['nse','zona','zona2']
lst_nse=['nse_mx','nse_br', 'nse_ar', 'nse_cl', 'nse_pe', 'nse_co']
lst_zona=['zona_mx','zona_br', 'zona_ar', 'zona_cl', 'zona_pe', 'zona_co']
lst_zona2=['zona2_mx','zona2_br', 'zona2_ar']
lst_sociodemograficas=['codpanelista','pais','sexo','edad','edadr', 'f05s','f05e']
lst_catalog=['key', 'respondentid', 'variable', 'valor', 'file']
lst_panel_ola_cuesti=['ola','cuesti','responsedate','endtime','duration','status','type','device','ponderado210609']

#Compila las variables sociodemográficas para el DF
lst_dim_demograficas=list(sum(zip(lst_pk_respondent+ lst_sociodemograficas+ lst_new_vars), ()))

#Compila las variables sociodemográficas que luego serán eliminadas
lst_black=list(sum(zip(lst_new_vars+lst_nse+ lst_zona+ lst_zona2+lst_sociodemograficas), ()))

# Defino la variables de cabecera y genero DF

lst_cabecera=list(sum(zip(lst_pk_respondent+ lst_panel_ola_cuesti+['file']), ()))

In [ ]:
## Base Sociodemográfica
# Agrupa variables sociodemográficas 
# Genera DF Sociodemográfico por Respondent-Key]
# Elimina las variables del Panel


#Genera las variables agrupadas
df_panel_final[lst_new_vars[0]] = df_panel_final[lst_nse].bfill(axis=1).iloc[:, 0]
df_panel_final[lst_new_vars[1]] = df_panel_final[lst_zona].bfill(axis=1).iloc[:, 0]
df_panel_final[lst_new_vars[2]] = df_panel_final[lst_zona2].bfill(axis=1).iloc[:, 0]

#Genera el DF de variables sociodemográficas por Respondentid - KEY
df_socioeconomico=df_panel_final[lst_dim_demograficas].drop_duplicates()

#Elimina las variables sociodemográficas del panel
df_panel_final.drop(columns=lst_black, inplace=True)

## Genero Cabecera de Panel
# Defino la variables de cabecera y genero DF

df_panel_cabecera=df_panel_final[lst_cabecera]
df_panel_cabecera['responsedate']=df_panel_cabecera['responsedate'].replace("NaN", np.NaN)
df_panel_cabecera['endtime']=df_panel_cabecera['endtime'].replace("NaN", np.NaN)
df_panel_cabecera['endtime'] = pd.to_datetime(df_panel_cabecera['endtime'], errors='coerce')
df_panel_cabecera['responsedate'] = pd.to_datetime(df_panel_cabecera['responsedate'], errors='coerce')
df_panel_cabecera['endtime'] = df_panel_cabecera['endtime'].fillna(df_panel_cabecera['responsedate'])
df_panel_cabecera['responsedate'] = df_panel_cabecera['responsedate'].fillna(df_panel_cabecera['endtime'])

#Elimina las variables ola_panel
lst_panel_ola_cuesti.remove('ola')
df_panel_final.drop(columns=lst_panel_ola_cuesti, inplace=True)

In [ ]:
## Generacion de fact Olas
def fn_denormalizacion(df):

    #Denormalización de Variables 
    df_panel_melt=pd.melt(df, id_vars=['key','respondentid','file'], var_name='variable', value_name='valor')
    ## Limpieza y filtrado de campos nulos
    df_panel_melt['valor']=df_panel_melt['valor'].replace(".", '')
    df_panel_melt['valor']=df_panel_melt['valor'].replace("NaN", np.NaN)
    df_panel_melt=df_panel_melt[df_panel_melt['valor']!='']
    df_panel_melt=df_panel_melt.dropna(subset=['valor'])
    df_panel_melt=df_panel_melt[df_panel_melt['variable']!='ola']
    return df_panel_melt

In [ ]:
lst=list(df_panel_final['ola'].drop_duplicates())
dataframes = []
for i in lst:
    df=df_panel_final[df_panel_final['ola']==i]
    df_panel_melt_tmp=fn_denormalizacion(df)
    dataframes.append(df_panel_melt_tmp)
    print(i)
df_panel_melt_final = pd.concat(dataframes, ignore_index=True)
df_panel_melt_final['valor'] = df_panel_melt_final['valor'].astype(str)
df_panel_melt_final['valor']=df_panel_melt_final['valor'].str.replace('|', '-') 
print(df_panel_melt_final.shape)

In [ ]:
##Insertar 
lst_df=[[df_socioeconomico,lst_dim_demograficas, 'df_sociodemografico', 'stg.fn_df_sociodemografico'],
        [df_panel_cabecera,lst_cabecera, 'df_panel_cabecera','stg.fn_df_panel_cabecera'] ,
        [df_panel_melt_final,lst_catalog,'df_panel','stg.fn_df_panel']        
        ]

In [ ]:
try:
    call_cursor()
    for e in range(len(lst_df)):
        df_resultado=lst_df[e][0]
        #df_resultado=df_resultado.iloc[:150000]
        lst_filtrada_eventos=lista_filtrada(df_resultado, lst_df[e][1])
        completa_cols_resultados(df_resultado, lst_df[e][1])

        ## Itera el DF
        # Separar el DataFrame en partes de 500,000 registros
        chunk_size = 200000
        num_chunks = -(-len(df_resultado) // chunk_size)  # Redondeo hacia arriba para calcular el número de partes

        # Exportar cada parte del DataFrame a un archivo de texto
        for i in range(num_chunks):
            hora_inicio1=(datetime.now())
            start_idx = i * chunk_size
            end_idx = min((i + 1) * chunk_size, len(df_resultado))
            df_chunk = df_resultado.iloc[start_idx:end_idx]
            copy_from_dataFile_StringIO(conn, df_chunk, lst_df[e][2],lst_df[e][1])
            print(i+1,' de ', num_chunks, ' ',(i+1)/num_chunks,'%')

        #Actualiza ambiente ODS    
        #call_upsert(lst_df[e][3])   
        print('Fin de upsert')      
             
except Exception as e:
    print(f"Error al procesar el archivo {i}: {e}")            

In [ ]:
hora_fin=(datetime.now())
delta=(hora_fin-hora_inicio).total_seconds()
print(hora_inicio,'-',hora_fin,'-', delta)

### Procesamiento de la Metadata

In [ ]:
def dict_to_df(data_dict):
    df_temp=pd.DataFrame.from_dict(data_dict, orient='index',columns=['valor'])
    df_temp['valor']= df_temp['valor'].str.lower()
    df_temp.index = df_temp.index.str.lower()
    df_temp['clave'] = df_temp.index
    df_temp.reset_index(drop=True, inplace=True)
    return df_temp


In [ ]:
lst_dict_redstat_type=[]
lst_dict_cols_names=[]
lst_dict_labels_type=[]
lst_dict_var2label=[]
lst_dict_var_opts=[]
for i in range(len(sav_files)):
    file=os.path.basename(sav_files[i])

    tmp=meta_lst[i].readstat_variable_types
    tmp=dict_to_df(tmp)
    tmp['file']=file
    lst_dict_redstat_type.append(tmp)

    tmp=meta_lst[i].column_names_to_labels
    tmp=dict_to_df(tmp)
    tmp['file']=file
    lst_dict_cols_names.append(tmp)

    tmp=meta_lst[i].original_variable_types
    tmp=dict_to_df(tmp)
    tmp['file']=file
    lst_dict_labels_type.append(tmp)

    tmp=meta_lst[i].variable_to_label
    tmp=dict_to_df(tmp)
    tmp['file']=file
    lst_dict_var2label.append(tmp)

    tmp=meta_lst[i].variable_value_labels
    lista_tuplas = [(k, sk, v) for k, subdict in tmp.items() for sk, v in subdict.items()]

    tmp = pd.DataFrame(lista_tuplas, columns=['pregunta', 'id_respuesta', 'respuesta'])
    tmp['id_respuesta']= tmp['id_respuesta'].astype(np.int64)
    tmp['pregunta']= tmp['pregunta'].str.lower()
    tmp['respuesta']= tmp['respuesta'].str.lower()    
    tmp['file']=file
    lst_dict_var_opts.append(tmp)

df_panel_final = pd.concat(dataframes, ignore_index=True)

df_dict_redstat_type= pd.concat(lst_dict_redstat_type, ignore_index=True)
df_dict_cols_names= pd.concat(lst_dict_cols_names, ignore_index=True)
df_dict_labels_type= pd.concat(lst_dict_labels_type, ignore_index=True)
df_dict_var2label= pd.concat(lst_dict_var2label, ignore_index=True)
df_catalogo_respuestas= pd.concat(lst_dict_var_opts, ignore_index=True)
print(df_catalogo_respuestas.shape)
#lst_meta_dicts=[dict_cols_names,dict_labels_type,dict_redstat_type,dict_var2label]


In [ ]:
df_metadata=pd.merge(df_catalogo_respuestas, df_dict_cols_names, left_on=['pregunta','file'], right_on=['clave','file'], how='left').drop(columns='clave')
df_metadata = df_metadata.rename(columns={'valor': 'texto_pregunta'})

df_metadata=pd.merge(df_metadata, df_dict_labels_type, left_on=['pregunta','file'], right_on=['clave','file'], how='left').drop(columns='clave')
df_metadata = df_metadata.rename(columns={'valor': 'label_type'})

df_metadata=pd.merge(df_metadata, df_dict_redstat_type, left_on=['pregunta','file'], right_on=['clave','file'], how='left').drop(columns='clave')
df_metadata = df_metadata.rename(columns={'valor': 'var_format_type'})

df_metadata=pd.merge(df_metadata, df_dict_var2label, left_on=['pregunta','file'], right_on=['clave','file'], how='left').drop(columns='clave')
df_metadata = df_metadata.rename(columns={'valor': 'label'})

df_metadata[['modulo', 'id_pregunta']] = df_metadata['pregunta'].str.split('#', n=1, expand=True)

In [ ]:
df_metadata.to_excel('fichero_metadata_full.xlsx',index=False)
print('export metadata')

lst_dict_var_opts=[]
for i in range(len(sav_files)):
    file=os.path.basename(sav_files[i])
    tmp_dict_var_opts=meta_lst[i].variable_value_labels
    lista_tuplas = [(k, sk, v) for k, subdict in tmp_dict_var_opts.items() for sk, v in subdict.items()]

    tmp_dict_var_opts = pd.DataFrame(lista_tuplas, columns=['pregunta', 'id_respuesta', 'respuesta'])
    tmp_dict_var_opts['id_respuesta']= tmp_dict_var_opts['id_respuesta'].astype(np.int64)
    tmp_dict_var_opts['pregunta']= tmp_dict_var_opts['pregunta'].str.lower()
    tmp_dict_var_opts['respuesta']= tmp_dict_var_opts['respuesta'].str.lower()    
    tmp_dict_var_opts['file']=file
    lst_dict_var_opts.append(tmp_dict_var_opts)

tmp_dict_var_opts=meta_lst[1].variable_value_labels
for k, subdict in tmp_dict_var_opts.items():
    print(k,' ','subdict') 
    for sk, v in subdict.items():
        print(sk,' ',v)

### Metadata de listado de respuestas por variable

## Catalogo de Valores posibles en cada uan de las respuestas
# Convertir el diccionario en una lista de tuplas (clave, subclave, valor)
lista_tuplas = [(k, sk, v) for k, subdict in dict_var_opts.items() for sk, v in subdict.items()]
# Crear un DataFrame a partir de la lista de tuplas
df_catalogo_respuestas = pd.DataFrame(lista_tuplas, columns=['pregunta', 'id_respuesta', 'respuesta'])
df_catalogo_respuestas['id_respuesta']= df_catalogo_respuestas['id_respuesta'].astype(np.int64)
df_catalogo_respuestas['pregunta']= df_catalogo_respuestas['pregunta'].str.lower()
df_catalogo_respuestas['respuesta']= df_catalogo_respuestas['respuesta'].str.lower()

#Catálogo de metadata de la metadata de preguntas y respuestas

df_dict_cols_names=dict_to_df(lst_meta_dicts[0])
df_dict_labels_type=dict_to_df(lst_meta_dicts[1])
df_dict_redstat_type=dict_to_df(lst_meta_dicts[2])
df_dict_var2label=dict_to_df(lst_meta_dicts[3])
